In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
df_train= pd.read_csv('train.data', sep='\t', header=None)
df_test= pd.read_csv('test.data', sep='\t', header=None)
df_dev= pd.read_csv('dev.data', sep='\t', header=None)
header_col = ['ID', 'Topic_name', 'S1', 'S2', 'Label', 'S1Tag', 'S2Tag']
df_train.columns = header_col
df_test.columns = header_col
df_dev.columns = header_col

In [2]:
def binary_label(df, lable_name):
    binary_label = []
    debateble = []
    for i in range(len(df)):
        if len(str(df[lable_name][i])) != 1:
            Yval = df[lable_name][i][1]
            Nval = df[lable_name][i][4]

            if int(Yval) > 2:
                binary_label.append(1)
            elif int(Yval) == 2:
                debateble.append(i)
            else:
                binary_label.append(0)
        else:
            score = int(df[lable_name][i])
            if score > 3:
                binary_label.append(1)
            elif score == 3:
                debateble.append(i)
            else:
                binary_label.append(0)

    df_f = df.drop(df.index[debateble])
    df_f[lable_name] = binary_label
    return df_f
df_f_train = binary_label(df_train, 'Label')
df_f_test = binary_label(df_test, 'Label')
df_f_dev = binary_label(df_dev, 'Label')
df_f_train['Label'].value_counts()
df_f_test['Label'].value_counts()
df_f_dev['Label'].value_counts()

0    2672
1    1470
Name: Label, dtype: int64

**Cosine Similarity**

In [6]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import SnowballStemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Stem the words
    words = [stemmer.stem(word) for word in words]
    # Join the words back into a string
    text = ' '.join(words)
    return text
df_f_train['S1'] = df_f_train['S1'].apply(preprocess_text)
df_f_train['S2'] = df_f_train['S2'].apply(preprocess_text)
df_f_test['S1'] = df_f_test['S1'].apply(preprocess_text)
df_f_test['S2'] = df_f_test['S2'].apply(preprocess_text)
df_f_dev['S1'] = df_f_dev['S1'].apply(preprocess_text)
df_f_dev['S2'] = df_f_dev['S2'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
df_f_train = df_f_train.reset_index()
df_f_test = df_f_test.reset_index()
df_f_dev = df_f_dev.reset_index()
#Function for calculating performance measures
def PerformanceMeasure(y_true,y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 score:", f1)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_sim_vectors(v1, v2):
    v1 = v1.reshape(1,-1)
    v2 = v2.reshape(1,-1)
    return cosine_similarity(v1, v2)[0][0]

def calculate_Cosi(df):
    Cosine_score = []
    for index in range(len(df)):
        s1 = df['S1'][index]
        s2 = df['S2'][index]
        s_all = [s1, s2]
        vectorizer = CountVectorizer().fit_transform(s_all)
        vectors = vectorizer.toarray()
        csim = cosine_similarity(vectors)
        Cosine_score.append(cosine_sim_vectors(vectors[0], vectors[1]))
    return Cosine_score
CosSimi_train = calculate_Cosi(df_f_train)
CosSimi_test = calculate_Cosi(df_f_test)
CosSimi_dev = calculate_Cosi(df_f_dev)
df_f_train['Baseline_Cosine_Similarity']=CosSimi_train
df_f_test['Baseline_Cosine_Similarity']=CosSimi_test
df_f_dev['Baseline_Cosine_Similarity']=CosSimi_dev
df_f_dev['Baseline_Cosine_Similarity'].mean()

0.3803346758769155

In [13]:
df_f_dev[:100]

,index,ID,Topic_name,S1,S2,Label,S1Tag,S2Tag,Baseline_Cosine_Similarity
0,0,17,A Walk To Remember,walk rememb definit true love,walk rememb im town im upset,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.316228
1,1,17,A Walk To Remember,walk rememb definit true love,walk rememb cutest thing,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214
2,2,17,A Walk To Remember,walk rememb definit true love,walk rememb abc famili your welcom,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.365148
3,3,17,A Walk To Remember,walk rememb definit true love,walk rememb amaz inspir,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214
4,4,17,A Walk To Remember,walk rememb definit true love,guy fave part walk rememb,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0.400000
...,...,...,...,...,...,...,...,...,...
95,114,72,Amber,amber alert get annoy,chill amber alert,1,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Can/O/MD/O/O we/O/JJ/B-NP/O chill/O/NN/I-NP/O ...,0.577350
96,115,72,Amber,amber alert get annoy,dam amber alert got phone scare hell,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Dam/O/NN/B-NP/O amber/O/NN/I-NP/O alert/O/NN/I...,0.377964
97,116,72,Amber,amber alert get annoy,everyon amber alert go zarap,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Everyone/O/NN/B-NP/O s/O/NNP/I-NP/O Amber/O/NN...,0.447214
98,117,72,Amber,amber alert get annoy,come never get amber alert,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,How/O/WRB/B-ADVP/O come/O/VBP/O/O I/O/PRP/B-NP...,0.670820


In [15]:
def cosine_baseline_threshold(df):
    X=df['Baseline_Cosine_Similarity']
    after_threshold=[]
    for i in range(len(X)):
        if X[i] >= 0.36:
            after_threshold.append(1)
        else:
            after_threshold.append(0)
    df['Baseline_Threshold_CS']=after_threshold
    return df


df_f_train=cosine_baseline_threshold(df_f_train)
df_f_test=cosine_baseline_threshold(df_f_test)
df_f_dev=cosine_baseline_threshold(df_f_dev)
#Evaluating Performance measures
print('Train:')
PerformanceMeasure(df_f_train['Label'],df_f_train['Baseline_Threshold_CS'])
print('\nTest:')
PerformanceMeasure(df_f_test['Label'],df_f_test['Baseline_Threshold_CS'])
print('\nDev:')
PerformanceMeasure(df_f_dev['Label'],df_f_dev['Baseline_Threshold_CS'])

Train:
Accuracy: 0.6295750216825672
Precision: 0.48032059539144123
Recall: 0.8398398398398398
F1 score: 0.611126286078485

Test:
Accuracy: 0.7040572792362768
Precision: 0.3954154727793696
Recall: 0.7885714285714286
F1 score: 0.5267175572519084

Dev:
Accuracy: 0.6419604056011589
Precision: 0.49697814969781495
Recall: 0.7272108843537415
F1 score: 0.5904446285556476


**TF-IDF**

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Converting Sentence 1 and Sentence 2 into list form
def Sent1Sent2_list(df):
    sent1 = []
    sent2 = []
    for i in range(len(df)):
        sent1.append(df['S1'][i])
        sent2.append(df['S2'][i])
    return sent1, sent2
# Embedding each sentence into a vector using TF_IDF and then computing cosine similarity of the two vectors
def TF_IDF_cosine_similarity(sent1,sent2):
#   TF_IDF_cosine_similarity_threshold =0.7
    sentence_list = [sent1,sent2]
    vect = TfidfVectorizer(stop_words="english")
    tfidf = vect.fit_transform(sentence_list)
    pairwise_similarity = tfidf * tfidf.T
    cosine_similarity = pairwise_similarity.toarray()[0][1]
    return cosine_similarity
#Computing cosine similarity score for TFIDF
def tf_idf_cosine_score(df):
    tf_idf_cosine=[]
    sent1,sent2=Sent1Sent2_list(df)
    for i in range(len(sent1)):
        tf_idf_cosine.append(TF_IDF_cosine_similarity(sent1[i],sent2[i]))
    return tf_idf_cosine

In [23]:
Train_tf_idf_cosine = tf_idf_cosine_score(df_f_train)
Test_tf_idf_cosine = tf_idf_cosine_score(df_f_test)
Dev_tf_idf_cosine = tf_idf_cosine_score(df_f_dev)
df_f_train['TF_IDF_Cosine_Similarity']=Train_tf_idf_cosine
df_f_test['TF_IDF_Cosine_Similarity']=Test_tf_idf_cosine
df_f_dev['TF_IDF_Cosine_Similarity']=Dev_tf_idf_cosine
df_f_dev['TF_IDF_Cosine_Similarity'].mean()
0.2670334779791318



def cosine_TDIDF_threshold(df):
    X=df['TF_IDF_Cosine_Similarity']
    after_threshold=[]
    for i in range(len(X)):
        if X[i] >= 0.26:
            after_threshold.append(1)
        else:
            after_threshold.append(0)
    df['TFIDF_Threshold_CS']=after_threshold
    return df


df_f_train=cosine_TDIDF_threshold(df_f_train)
df_f_test=cosine_TDIDF_threshold(df_f_test)
df_f_dev=cosine_TDIDF_threshold(df_f_dev)

In [24]:
df_f_dev[:100]

,index,ID,Topic_name,S1,S2,Label,S1Tag,S2Tag,Baseline_Cosine_Similarity,Baseline_Threshold_CS,TF_IDF_Cosine_Similarity,TFIDF_Threshold_CS
0,0,17,A Walk To Remember,walk rememb definit true love,walk rememb im town im upset,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.316228,0,0.190874,0
1,1,17,A Walk To Remember,walk rememb definit true love,walk rememb cutest thing,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214,1,0.291219,1
2,2,17,A Walk To Remember,walk rememb definit true love,walk rememb abc famili your welcom,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.365148,1,0.252334,0
3,3,17,A Walk To Remember,walk rememb definit true love,walk rememb amaz inspir,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214,1,0.291219,1
4,4,17,A Walk To Remember,walk rememb definit true love,guy fave part walk rememb,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0.400000,1,0.291219,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,114,72,Amber,amber alert get annoy,chill amber alert,1,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Can/O/MD/O/O we/O/JJ/B-NP/O chill/O/NN/I-NP/O ...,0.577350,1,0.503103,1
96,115,72,Amber,amber alert get annoy,dam amber alert got phone scare hell,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Dam/O/NN/B-NP/O amber/O/NN/I-NP/O alert/O/NN/I...,0.377964,1,0.291069,1
97,116,72,Amber,amber alert get annoy,everyon amber alert go zarap,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,Everyone/O/NN/B-NP/O s/O/NNP/I-NP/O Amber/O/NN...,0.447214,1,0.411207,1
98,117,72,Amber,amber alert get annoy,come never get amber alert,0,That/O/DT/B-NP/O amber/O/NN/I-NP/O alert/O/NN/...,How/O/WRB/B-ADVP/O come/O/VBP/O/O I/O/PRP/B-NP...,0.670820,1,0.503103,1


In [25]:
print('Train:')
PerformanceMeasure(df_f_train['Label'],df_f_train['TFIDF_Threshold_CS'])
print('\nTest:')
PerformanceMeasure(df_f_test['Label'],df_f_test['TFIDF_Threshold_CS'])
print('\nDev:')
PerformanceMeasure(df_f_dev['Label'],df_f_dev['TFIDF_Threshold_CS'])

Train:
Accuracy: 0.6550737207285342
Precision: 0.5014653709702298
Recall: 0.8135635635635635
F1 score: 0.6204790533447847

Test:
Accuracy: 0.6897374701670644
Precision: 0.37822349570200575
Recall: 0.7542857142857143
F1 score: 0.5038167938931297

Dev:
Accuracy: 0.6588604538870111
Precision: 0.5147439213657528
Recall: 0.6768707482993197
F1 score: 0.5847781369379959


**SBERT**

In [26]:
!pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 2.5 MB/s eta 0:00:00


In [27]:
from sentence_transformers import SentenceTransformer
import torch
#Loading the sbert model
sbert_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
def Sent1Sent2_list(df):
    sent1 = []
    sent2 = []
    for i in range(len(df['S1'])):
        sent1.append(df['S1'][i])
        sent2.append(df['S2'][i])
    return sent1,sent2
#Extracting Embeddings from sbert model
def compute_sbert_embedding(df):
    sent1,sent2=Sent1Sent2_list(df)
    sentence_embeddings1 = sbert_model.encode(sent1,show_progress_bar=True)
    sentence_embeddings2 = sbert_model.encode(sent2,show_progress_bar=True)
    return sentence_embeddings1,sentence_embeddings2
#Function for calculating cosine score
def cosine(u,v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
#Compute cosine-similarities using embeddings
def Sbert_cosine_similarity(df):
    result=[]
    sentence_embeddings1,sentence_embeddings2=compute_sbert_embedding(df)
    for i in range(len(sentence_embeddings1)):
        Sbert_Cosine_Similarity = cosine(sentence_embeddings1[i],sentence_embeddings2[i])
        result.append(Sbert_Cosine_Similarity)
    return result

In [29]:
Sbert_CosSimi_train = Sbert_cosine_similarity(df_f_train)
Sbert_CosSimi_test = Sbert_cosine_similarity(df_f_test)
Sbert_CosSimi_dev = Sbert_cosine_similarity(df_f_dev)

Batches:   0%|          | 0/361 [00:00<?, ?it/s]

Batches:   0%|          | 0/361 [00:00<?, ?it/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

Batches:   0%|          | 0/27 [00:00<?, ?it/s]

Batches:   0%|          | 0/130 [00:00<?, ?it/s]

Batches:   0%|          | 0/130 [00:00<?, ?it/s]

In [31]:
df_f_train['Sbert_Cosine_Similarity']=Sbert_CosSimi_train
df_f_test['Sbert_Cosine_Similarity']=Sbert_CosSimi_test
df_f_dev['Sbert_Cosine_Similarity']=Sbert_CosSimi_dev
df_f_dev['Sbert_Cosine_Similarity'].mean()

0.5484643

In [34]:
def cosine_sbert_threshold(df):
    X=df['Sbert_Cosine_Similarity']
    after_threshold=[]
    for i in range(len(X)):
        if X[i] >= 0.57:
            after_threshold.append(1)
        else:
            after_threshold.append(0)
    df['Sbert_Threshold_CS']=after_threshold
    return df
cosine_sbert_threshold(df_f_train)
cosine_sbert_threshold(df_f_test)
cosine_sbert_threshold(df_f_dev)

,index,ID,Topic_name,S1,S2,Label,S1Tag,S2Tag,Baseline_Cosine_Similarity,Baseline_Threshold_CS,TF_IDF_Cosine_Similarity,TFIDF_Threshold_CS,Sbert_Cosine_Similarity,Sbert_Threshold_CS
0,0,17,A Walk To Remember,walk rememb definit true love,walk rememb im town im upset,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.316228,0,0.190874,0,0.411617,0
1,1,17,A Walk To Remember,walk rememb definit true love,walk rememb cutest thing,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214,1,0.291219,1,0.550082,0
2,2,17,A Walk To Remember,walk rememb definit true love,walk rememb abc famili your welcom,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.365148,1,0.252334,0,0.411227,0
3,3,17,A Walk To Remember,walk rememb definit true love,walk rememb amaz inspir,1,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0.447214,1,0.291219,1,0.463302,0
4,4,17,A Walk To Remember,walk rememb definit true love,guy fave part walk rememb,0,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0.400000,1,0.291219,1,0.498624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4137,4722,1902,iTunes,first podcast drop itun,wonder spend much itun,0,My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...,0.250000,0,0.201993,0,0.313820,0
4138,4723,1902,iTunes,first podcast drop itun,dead gone ep itun,0,My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0.250000,0,0.170776,0,0.477973,0
4139,4724,1902,iTunes,first podcast drop itun,amslingshot 7 itun chart,0,My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0.288675,0,0.201993,0,0.162879,0
4140,4725,1902,iTunes,first podcast drop itun,itun delet voic memo,0,My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...,0.250000,0,0.170776,0,0.242315,0


In [35]:
print('Train:')
PerformanceMeasure(df_f_train['Label'],df_f_train['Sbert_Threshold_CS'])
print('\nTest:')
PerformanceMeasure(df_f_test['Label'],df_f_test['Sbert_Threshold_CS'])
print('\nDev:')
PerformanceMeasure(df_f_dev['Label'],df_f_dev['Sbert_Threshold_CS'])

Train:
Accuracy: 0.7120555073720729
Precision: 0.5582557738710789
Recall: 0.8105605605605606
F1 score: 0.6611553378240457

Test:
Accuracy: 0.7744630071599046
Precision: 0.4734848484848485
Recall: 0.7142857142857143
F1 score: 0.5694760820045558

Dev:
Accuracy: 0.6803476581361662
Precision: 0.5401982378854625
Recall: 0.6673469387755102
F1 score: 0.5970785149117469
